##### appends species sources root
###### so that manny can be imported as a module

In [ ]:
import sys
nb_dir = '../..'
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

In [ ]:
from species.modeling.manny import *

In [ ]:
# Bokeh
from bokeh.io import output_notebook

output_notebook()

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

## User-defined inputs

GENERAL INPUTS:

In [ ]:
#'image_file' -- filepath to image from which spectra will be extracted
# image_file = r"W:\Portland_General_2019_034492\03_ANALYSIS\20190724M1\02_MOSAICS\190724M1_REFbnPCA10_bnMNF10_bn_PSRI_PRI_NDVI_REFir_bnir_dis_entropy_CHM"
# image_file = r"U:\users\Luke_Brown\umap_interactive\724m1_pgen_noMNF_subset"
image_file = r"U:\PG_E_Midcycle_IVM_Pilot\03_ANALYSIS\626_B_North\02_MOSAICS\626B_North_Layer_Stack_BN39_MNF30_PCA30_VEG10_DR8_VAR8"

#'shape_file' -- filepath to shapefile containing focal features (eg. tree polygons)
# shape_file = r"W:\00_HSI_Training_Data\03_Training_Data_KL\03_ANALYSIS\190724M1\01_REFERENCE_VECTORS\02_TRAIN_VAL_POLYGONS\03_TRAINING_POLYGONS\PGE_20190724_Training_Polys_20220311_AS.shp"
shape_file = r"D:\Projects_HSI\pg_e_ivm_hsi_2020\626B_North\tv_polys\PG_E_IVM_MFGH_626M1B_TVPolys_NORTH_FINAL.shp"

#'class_field' -- name of feature class field listing species class code (numeric)
# class_field = 'Code_3'
class_field = 'Class_9'

#'class_lookup' -- filepath to csv listing feature classes (eg. tree species) and associated class codes
# class_lookup = r"W:\00_HSI_Training_Data\03_Training_Data_KL\03_ANALYSIS\190724M1\01_REFERENCE_VECTORS\02_TRAIN_VAL_POLYGONS\Portland_General_2019_Tree_Code_Lookup_190724M1_code_3.csv"
class_lookup = r"U:\\PG_E_Midcycle_IVM_Pilot\\00_PRELIM\\GROUND_TRUTH_DATA\\PG_E_IVM_Pilot_Tree_Code_Lookup_TEMPLATE.csv"

# 'chips' -- if 'True', outputs individual chips (as tiffs) in location [wd]\chips
chips = False

#### Let's get some stats
Reading in your data and returning sample size by species

In [ ]:
species_d = parse_lookup_table(class_lookup)

chip_d, chip_d_ss, species_ct_d = extract_chips(image_file, shape_file, class_field, species_d, chip=chips)
print("\nInput shapefile contains:")
for k, v in species_ct_d.items():
    print(f"{v} features of species {k}")

### CHOOSE YOUR PLOTS!
Plot parameters and other user input are specified here

#### Spectral Signature Plot (ss_plot) ####
inputs: n-band image, polygon set and class lookup table
outputs: a plot of spectral signature for each class
users can specify: whether plot should include standard deviation ribbons for species signatures; which bands and species to use (defaults: all bands, all species).

*If the input image includes derived bands (MNF, textures), the user *should** specify a subset of bands that exclude these.*

In [ ]:
# 'ss_plot' -- If 'true', output will include a spectral signatures plot
ss_plot = True

# 'st_dev' -- If 'True', plot will include st dev ribbons around the mean spectral value for each class
st_dev = False

# 'band_select_ss' -- list of bands to include in Spectral Signature visualization. If 'None', all bands will be used
#band_select_ss = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]

# 'ss_fp' -- filepath for saving the spectral signatures plot output. If 'None', will not save to file
ss_fp = None

#### Spectral Pair Plot (pp_plot) ####
inputs:
outputs:
users can specify:

In [ ]:
# 'pp_plot' -- If 'true', output will include a spectral pairplot for up to 10 spectral bands
pp_plot = True

# 'band_select_pp' -- list of bands to include in pairplot comparison; choose maximum of 10 bands.
band_select_pp = [ 96, 98, 100, 102, 104, 106]

# 'pp_fp' -- filepath for saving the output pairplot. If 'None', will not save to file
pp_fp = None

#### UMAP Visualization Plot (u_plot) ####
inputs:
outputs:
users can specify:

## Let's plot!
Functions are called in this portion of the script

In [ ]:
# todo: clean this up

In [ ]:
# 'u_plot' -- If 'true', output will include a UMAP plot
u_plot = True

if pp_plot or u_plot:
    px_df = pixel_df(chip_d, species_d)

In [ ]:
# 'band_select_u' -- list of bands to include in UMAP visualization. If 'None', all bands will be used
band_select_u = 'all'  # [1,2,3,4,5,51,52,53,86,87,88,89,96,97,98,99,100,101,102,103,104,105,106]

if band_select_u == 'all':
    with rasterio.open(image_file) as src:
        band_select_u = list(range(1, src.count + 1))

# 'species_select_u' -- list of species class codes (numeric) corresponding to species you wish to include
# in UMAP visualization. If 'None', all species will be used
# Examples:
# species_select_u = [2, 4, 25, 27, 31], or
# [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
species_select_u = px_df.class_code.unique()

# 'u_fp' -- filepath for saving the output UMAP plot. If 'None', will not save to file
u_fp = None

In [ ]:
if ss_plot:
    print('Preparing Spectral Signature Plot')
    spectral_signature_plot(chip_d_ss, species_d, stdev=st_dev, ss_fp=ss_fp)

In [ ]:
if pp_plot:
    print('Preparing Spectral Pairplot')
    spectral_pairplot(band_select_pp, px_df, pp_fp=pp_fp)

In [ ]:
if u_plot:
    print('Preparing UMAP Visualization Plot for the following species:' + '\n')
    species_select = list(map(species_d.get, species_select_u))
    print(species_select, '\n')

    pix_df = umap_vis_plot(px_df, shape_file, band_select_u, species_select, u_fp=u_fp)
    print('Code has finished, so hopefully you saw some plots')

In [ ]:
# 'ss_fp' -- filepath for saving the spectral signatures plot output. If 'None', will not save to file
ss_fp = None

# 'pp_fp' -- filepath for saving the output pairplot. If 'None', will not save to file
pp_fp = None

# 'u_fp' -- filepath for saving the output UMAP plot. If 'None', will not save to file
u_fp = None